In [1]:
import pickle

with open("/kaggle/input/mai-processed/X.pickle", "rb") as p:
    X = pickle.load(p)
with open("/kaggle/input/mai-processed/y.pickle", "rb") as p:
    y = pickle.load(p)

In [2]:
print(len(X))
print(len(y))

824
824


In [3]:
mai = {}
t = []
l = []
for text, label in zip(X, y):
    t.append(text)
    l.append(label)
mai["text"] = t
mai["label"] = l
assert(len(mai["text"]) == len(mai["label"]))

In [4]:
!pip install datasets

     |████████████████████████████████| 325 kB 884 kB/s            
     |████████████████████████████████| 212 kB 14.7 MB/s            


In [5]:
from datasets import Dataset

mai_data = Dataset.from_dict(mai)

In [6]:
import gc

del X
del y
gc.collect()

68

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot_small-90M")

Downloading:   0%|          | 0.00/205 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/941k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/337k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [12]:
def tok(data):
    text = data["text"]
    label = data["label"]
    t_text = tokenizer(text, padding="max_length", truncation=True, return_tensors="tf")
    t_label = tokenizer(label, padding="max_length", truncation=True, return_tensors="tf")
    output = {'input_ids': t_text["input_ids"][0], 'attention_mask': t_text["attention_mask"][0], 'labels': t_label["input_ids"][0], 'decoder_attention_mask': t_label["attention_mask"][0]}
    return output

tokenized_mai = mai_data.map(tok)

  0%|          | 0/824 [00:00<?, ?ex/s]

In [14]:
import tensorflow as tf

def to_tf_dataset(dataset):  
    columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']
    dataset.set_format(type='tensorflow', columns=columns)
    return_types = {'input_ids':tf.int32, 'attention_mask':tf.int32, 
                'labels':tf.int32, 'decoder_attention_mask':tf.int32,  }
    return_shapes = {'input_ids': tf.TensorShape([None]), 'attention_mask': tf.TensorShape([None]), 
                  'labels': tf.TensorShape([None]), 'decoder_attention_mask':tf.TensorShape([None])}
    ds = tf.data.Dataset.from_generator(lambda : dataset, return_types, return_shapes)
    return ds

tf_mai = to_tf_dataset(tokenized_mai)

In [15]:
tf_mai = tf_mai.batch(8)

In [19]:
import tensorflow_addons as tfa
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot_small-90M")

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5))

All model checkpoint layers were used when initializing TFBlenderbotSmallForConditionalGeneration.

All the layers of TFBlenderbotSmallForConditionalGeneration were initialized from the model checkpoint at facebook/blenderbot_small-90M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBlenderbotSmallForConditionalGeneration for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as the 'labels' key of the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [20]:
model.fit(tf_mai, epochs=3)

Epoch 1/3
103/103 [==============================] - 105s 759ms/step - loss: 4.3400
Epoch 2/3
103/103 [==============================] - 78s 759ms/step - loss: 3.7569
Epoch 3/3
103/103 [==============================] - 78s 759ms/step - loss: 3.4470


In [21]:
model.save_pretrained("/kaggle/working")

In [26]:
input_text = "I have trouble managing my time and it has caused me to perform poorly academically. How can I manage my time better?"
encoded_query = tokenizer(input_text, return_tensors="tf", pad_to_max_length=True, truncation=True)
input_ids = encoded_query["input_ids"]
attention_mask = encoded_query["attention_mask"]
answer = model.generate(input_ids, attention_mask=attention_mask)
decoded_answer = tokenizer.decode(answer.numpy()[0])
print(decoded_answer)

__start__ i'm sorry to hear that you're struggling with time management. it sounds like you've had a difficult time managing your time and that's understandable. it's also important to remember that time management is a skill that requires practice and practice. if you practice a lot of time, you'll be able to learn more about how to manage your time better. practice makes perfect. the more practice you have, the easier it will be to master your time management skills. practice is the first step. practice will help you get better at managing time management, but practice will also give you more time to practice
